In [1]:
"""
CASMO: Confident Adaptive Selective Momentum Optimizer

A production-ready PyTorch optimizer that extends Adam with confidence-based learning rate scaling.

Core Innovation: AGAR (Adaptive Gradient Alignment Ratio)
    AGAR = ||E[g]||² / (||E[g]||² + Var[g])
    
    Measures signal (consistent gradient direction) vs noise (random fluctuations).
    Naturally ranges from 0 (pure noise) to 1 (pure signal) for interpretable confidence metrics.

Performance:
    - Faster than AdamW on large models (-2% overhead with per-group mode)
    - Configurable granularity for speed/precision tradeoff
    - Pre-allocated buffers eliminate allocation overhead

Usage Example:
    >>> from casmo import CASMO
    >>> optimizer = CASMO(model.parameters(), lr=1e-3, weight_decay=0.01)
    >>> for epoch in range(num_epochs):
    ...     for batch in dataloader:
    ...         loss = model(batch)
    ...         loss.backward()
    ...         optimizer.step()
    ...         optimizer.zero_grad()

Reference: 
    Kingma & Ba (2015). "Adam: A Method for Stochastic Optimization"
    https://arxiv.org/abs/1412.6980
"""

from typing import Tuple, Optional, Callable, Dict, Any
import torch
import numpy as np
from collections import deque
import logging


class DDEAdapter:
    """
    Drift-Detecting EMA adapter for tau threshold adjustment.
    
    Tracks AGAR variance to adaptively adjust tau while preventing
    runaway adaptation to noise or memorization signals.
    O(1) memory and compute per step.
    """
    
    # EMA update rates
    EMA_MEAN_RATE = 0.001
    EMA_VAR_DECAY = 0.99
    EMA_VAR_RATE = 0.01
    
    # Adaptive gain bounds
    MIN_GAIN = 0.001
    MAX_GAIN = 0.01
    GAIN_SCALE = 0.1
    
    # Memorization detection threshold
    MEMORIZATION_FACTOR = 1.2
    
    def __init__(self, tau_init: float, tau_clip_range: Tuple[float, float], 
                 dead_zone_factor: float = 0.2):
        """
        Initialize the DDE adapter.
        
        Args:
            tau_init: Initial tau value
            tau_clip_range: (min, max) bounds for tau
            dead_zone_factor: Ignore deviations smaller than this fraction of tau.
                Prevents chasing noise. Default: 0.2 (20%)
        """
        self.tau = tau_init
        self.tau_calibrated: Optional[float] = None
        self.clip_range = tau_clip_range
        self.dead_zone = dead_zone_factor
        
        # EMA state for variance tracking
        self.mean_agar = tau_init
        self.ema_var = 0.01
    
    def update(self, agar_value: float) -> float:
        """
        Update tau threshold using variance-adaptive gain and dead zone filtering.
        
        Args:
            agar_value: Current AGAR measurement
            
        Returns:
            Updated tau value (clipped to valid range)
        """
        # Update EMA mean
        diff = agar_value - self.mean_agar
        self.mean_agar += self.EMA_MEAN_RATE * diff
        
        # Update EMA variance: Var[X] = E[(X - μ)²]
        self.ema_var = self.EMA_VAR_DECAY * self.ema_var + self.EMA_VAR_RATE * (diff ** 2)
        
        # Relative variance (scale-invariant)
        rel_var = self.ema_var / (self.mean_agar + 1e-8)
        
        # Prevent tau from chasing memorization signals
        if self.tau_calibrated is not None and agar_value > self.MEMORIZATION_FACTOR * self.tau_calibrated:
            # AGAR suspiciously high - likely overfitting, freeze tau
            return self.tau
        
        # Dead zone: only adapt if deviation exceeds threshold
        dead_zone_reference = self.tau_calibrated if self.tau_calibrated is not None else self.tau
        deviation = abs(agar_value - self.tau)
        if deviation > self.dead_zone * dead_zone_reference:
            # Variance-adaptive gain: higher variance → faster adaptation
            alpha = self.MIN_GAIN + min(rel_var * self.GAIN_SCALE, self.MAX_GAIN - self.MIN_GAIN)
            new_tau = (1 - alpha) * self.tau + alpha * agar_value
            
            # Never decrease tau below calibrated baseline
            if self.tau_calibrated is not None:
                new_tau = max(new_tau, self.tau_calibrated)
            
            self.tau = new_tau
        
        return float(np.clip(self.tau, self.clip_range[0], self.clip_range[1]))


class CASMO(torch.optim.Optimizer):
    """
    Confident Adaptive Selective Momentum Optimizer.
    
    Extends Adam with confidence-based learning rate scaling using AGAR metrics.
    Automatically adapts to gradient signal-to-noise ratio for improved convergence.
    
    Uses universal sigmoid-based confidence mapping that adapts to any noise distribution:
    - Clean data: High confidence baseline
    - Pervasive noise: Adaptive scaling with high c_min
    - Mixed batches: Strong discrimination via distribution statistics
    
    Args:
        params (iterable): Iterable of parameters to optimize or dicts defining parameter groups
        lr (float, optional): Learning rate. Default: 1e-3
        betas (Tuple[float, float], optional): Coefficients for computing running averages 
            of gradient and its square (β₁, β₂). Default: (0.9, 0.999)
        eps (float, optional): Term added to denominator for numerical stability. Default: 1e-8
        weight_decay (float, optional): Decoupled weight decay coefficient (AdamW-style). 
            Default: 0.0
        tau_init_steps (int, optional): Number of initial steps to collect AGAR samples 
            for automatic threshold calibration. Must be >= 50. Default: 500
            Recommended: max(500, int(0.05 * total_steps))
        tau_clip_range (Tuple[float, float], optional): Min/max bounds for tau threshold. 
            Default: (0.01, 0.5)
        tau_dead_zone (float, optional): Dead zone factor for tau adaptation.
            Ignores AGAR deviations smaller than this fraction of tau to prevent chasing noise.
            Default: 0.2 (20%)
        c_min (float, optional): Minimum confidence scaling factor to prevent learning rate 
            from becoming too small. Must be in [0, 1]. Default: 0.1
            Note: After calibration, c_min is automatically computed based on noise level.
        granularity (str, optional): AGAR computation granularity.
            - 'parameter': Per-parameter confidence scaling (~13% overhead on large models).
              Use for small models (<10M params) or when layer-specific adaptation matters.
            - 'group': Per-group confidence scaling (faster than AdamW on large models).
              Recommended for production use, large models (>10M params), and hyperparameter sweeps.
            Default: 'group'
        agar_clamp_factor (float, optional): Outlier clamping factor for AGAR computation.
            Clamps moment estimates to ±(mean * factor) to handle extreme values.
            Set to None to disable clamping. Default: 10.0
        log_level (int, optional): Logging verbosity. 0=silent, 1=errors, 2=warnings, 
            3=info. Default: 1
    
    Raises:
        ValueError: If any parameter is outside its valid range
        RuntimeError: If NaN or Inf gradients are detected during optimization
        NotImplementedError: If sparse gradients are encountered
    
    Note:
        This optimizer does not support sparse gradients. Use torch.optim.SparseAdam
        for sparse gradient scenarios.
    
    Example:
        >>> model = YourModel()
        >>> optimizer = CASMO(model.parameters(), lr=1e-3, weight_decay=0.01)
        >>> 
        >>> for epoch in range(num_epochs):
        ...     for batch in dataloader:
        ...         optimizer.zero_grad()
        ...         loss = model(batch)
        ...         loss.backward()
        ...         optimizer.step()
    """
    
    def __init__(
        self,
        params,
        lr: float = 1e-3,
        betas: Tuple[float, float] = (0.9, 0.999),
        eps: float = 1e-8,
        weight_decay: float = 0.0,
        tau_init_steps: int = 500,
        tau_clip_range: Tuple[float, float] = (0.01, 0.5),
        tau_dead_zone: float = 0.2,  # Large dead zone to prevent chasing memorization
        c_min: float = 0.1,
        granularity: str = 'group',
        agar_clamp_factor: Optional[float] = 10.0,
        log_level: int = 1,
    ):
        if not 0.0 <= lr:
            raise ValueError(f"Invalid learning rate: {lr}")
        if not 0.0 <= eps:
            raise ValueError(f"Invalid epsilon: {eps}")
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError(f"Invalid beta1: {betas[0]}")
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError(f"Invalid beta2: {betas[1]}")
        if not 0.0 <= weight_decay:
            raise ValueError(f"Invalid weight_decay: {weight_decay}")
        if not 0.0 <= c_min <= 1.0:
            raise ValueError(f"Invalid c_min: {c_min}")
        if tau_init_steps < 50:
            raise ValueError(f"tau_init_steps too small: {tau_init_steps} (minimum: 50)")
        if not 0.0 <= tau_dead_zone <= 1.0:
            raise ValueError(f"Invalid tau_dead_zone: {tau_dead_zone} (must be in [0, 1])")
        if granularity not in ['parameter', 'group']:
            raise ValueError(f"Invalid granularity: {granularity} (must be 'parameter' or 'group')")
        
        defaults = dict(
            lr=lr,
            betas=betas,
            eps=eps,
            weight_decay=weight_decay,
            tau_init_steps=tau_init_steps,
            tau_clip_range=tau_clip_range,
            tau_dead_zone=tau_dead_zone,
            c_min=c_min,
            granularity=granularity,
            agar_clamp_factor=agar_clamp_factor,
        )
        
        super().__init__(params, defaults)
        
        # Setup logging
        self.logger = logging.getLogger('CASMO')
        if not self.logger.handlers:
            handler = logging.StreamHandler()
            handler.setFormatter(logging.Formatter('[CASMO] %(message)s'))
            self.logger.addHandler(handler)
        self.logger.setLevel(self._get_log_level(log_level))
        
        self._step_count = 0
        
        # Initialize per-group state for tau calibration and buffer reuse
        self._group_states: Dict[int, Dict[str, Any]] = {}
        for idx, group in enumerate(self.param_groups):
            group_tau_dead_zone = group.get('tau_dead_zone', tau_dead_zone)
            group_tau_clip_range = group.get('tau_clip_range', tau_clip_range)
            group_tau_init_steps = group.get('tau_init_steps', tau_init_steps)
            
            self._group_states[idx] = {
                'tau_adapter': DDEAdapter(1.0, group_tau_clip_range, dead_zone_factor=group_tau_dead_zone),
                'tau_initialized': False,
                'agar_buffer': deque(maxlen=group_tau_init_steps),
                'reuse_buffer_exp_avg': None,
                'reuse_buffer_exp_avg_sq': None,
                'current_agar': None,
                'current_confidence': None,
                'agar_mean': None,
                'agar_std': None,
                'agar_median': None,
                'agar_p10': None,
                'agar_p90': None,
                'c_min': c_min,
            }
    
    def _get_log_level(self, level: int) -> int:
        """
        Convert custom log level to Python logging level.
        
        Args:
            level: Custom level (0=silent, 1=error, 2=warning, 3=info)
            
        Returns:
            Python logging level constant
        """
        level_map = {
            0: logging.CRITICAL + 1,  # Silent
            1: logging.ERROR,
            2: logging.WARNING,
            3: logging.INFO,
        }
        return level_map.get(level, logging.WARNING)
    
    def _log(self, level: int, message: str) -> None:
        """
        Internal logging utility using Python logging module.
        
        Args:
            level: Message severity level (1=error, 2=warning, 3=info)
            message: Log message to output
        """
        if level == 1:
            self.logger.error(message)
        elif level == 2:
            self.logger.warning(message)
        elif level == 3:
            self.logger.info(message)
    
    def _validate_gradient(self, grad: torch.Tensor, group_idx: int) -> None:
        """
        Validate gradient for NaN, Inf, and sparse tensors.
        
        Args:
            grad: Gradient tensor to validate
            group_idx: Parameter group index for error messages
            
        Raises:
            RuntimeError: If NaN or Inf detected
            NotImplementedError: If sparse gradient detected
        """
        if torch.isnan(grad).any():
            raise RuntimeError(
                f"NaN gradient detected in parameter group {group_idx}. "
                "Consider using gradient clipping: torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)"
            )
        if torch.isinf(grad).any():
            raise RuntimeError(
                f"Inf gradient detected in parameter group {group_idx}. "
                "Check for numerical overflow in loss computation or model outputs."
            )
        if grad.is_sparse:
            raise NotImplementedError(
                "CASMO does not support sparse gradients. "
                "Use torch.optim.SparseAdam for sparse gradient scenarios, "
                "or convert gradients to dense format with grad.to_dense()."
            )
    
    def _init_param_state(self, p: torch.Tensor) -> Dict[str, Any]:
        """
        Initialize optimizer state for a parameter.
        
        Args:
            p: Parameter tensor
            
        Returns:
            Initialized state dictionary with step counter and moment estimates
        """
        state = self.state[p]
        if len(state) == 0:
            state['step'] = 0
            state['exp_avg'] = torch.zeros_like(p, memory_format=torch.preserve_format)
            state['exp_avg_sq'] = torch.zeros_like(p, memory_format=torch.preserve_format)
        return state
    
    def _update_moments(self, state: Dict[str, Any], grad: torch.Tensor, beta1: float, beta2: float) -> None:
        """
        Update exponential moving averages of gradient moments.
        
        Args:
            state: Parameter state dictionary
            grad: Current gradient
            beta1: First moment decay rate (β₁)
            beta2: Second moment decay rate (β₂)
        """
        exp_avg = state['exp_avg']
        exp_avg_sq = state['exp_avg_sq']
        state['step'] += 1
        
        exp_avg.mul_(beta1).add_(grad, alpha=1 - beta1)
        exp_avg_sq.mul_(beta2).addcmul_(grad, grad, value=1 - beta2)
    
    def _apply_weight_update(self, p: torch.Tensor, state: Dict[str, Any], lr: float, 
                            weight_decay: float, eps: float, confidence: torch.Tensor,
                            beta1: float, beta2: float) -> None:
        """
        Apply Adam-style parameter update with confidence-scaled learning rate.
        
        Implements decoupled weight decay (AdamW) with bias-corrected moments
        and confidence-based learning rate modulation.
        
        Args:
            p: Parameter tensor to update
            state: Parameter state dictionary containing moments
            lr: Base learning rate
            weight_decay: Decoupled weight decay coefficient
            eps: Numerical stability constant (ε)
            confidence: Confidence scaling factor in [c_min, 1.0]
            beta1: First moment decay rate (β₁)
            beta2: Second moment decay rate (β₂)
        """
        exp_avg = state['exp_avg']
        exp_avg_sq = state['exp_avg_sq']
        step = state['step']
        
        # Bias correction
        bias_correction1 = 1 - beta1 ** step
        bias_correction2 = 1 - beta2 ** step
        m_hat = exp_avg / bias_correction1
        v_hat = exp_avg_sq / bias_correction2
        
        # Weight decay (decoupled)
        if weight_decay != 0:
            p.mul_(1 - lr * weight_decay)
        
        # Apply update with confidence-scaled learning rate
        denom = v_hat.sqrt().add_(eps)
        step_size = lr * confidence
        p.addcdiv_(m_hat, denom, value=-step_size)
    
    def _compute_agar(
        self,
        exp_avg: torch.Tensor,
        exp_avg_sq: torch.Tensor,
        eps: float,
        clamp_factor: Optional[float],
    ) -> torch.Tensor:
        """
        Compute Adaptive Gradient Alignment Ratio (AGAR) from exponential moving averages.
        
        AGAR quantifies the signal-to-noise ratio in gradients:
            AGAR = mean(signal / (signal + noise))
            where signal = (E[g])² (squared mean gradient per element)
                  noise = Var[g] = E[g²] - (E[g])² (gradient variance per element)
        
        Args:
            exp_avg (torch.Tensor): Exponential moving average of gradients (first moment)
            exp_avg_sq (torch.Tensor): Exponential moving average of squared gradients (second moment)
            eps (float): Small constant for numerical stability
            clamp_factor (Optional[float]): Outlier clamping factor (None to disable)
        
        Returns:
            torch.Tensor: Scalar AGAR value in range [0, 1], where:
                - 0 indicates pure noise (no consistent gradient direction)
                - 1 indicates pure signal (perfectly consistent gradients)
        
        Note:
            AGAR is computed per-element then uniformly averaged across all elements.
            This provides robustness across parameters with different scales.
            Uses raw moments to preserve the variance relationship Var[g] = E[g²] - (E[g])².
            Bias correction would distort this relationship and cause AGAR instability.
        """
        # Outlier protection: clamp extreme values based on gradient statistics
        if clamp_factor is not None:
            m_scale = exp_avg.abs().mean() + eps
            v_scale = exp_avg_sq.mean() + eps
            m_clamped = torch.clamp(exp_avg, min=-m_scale * clamp_factor, max=m_scale * clamp_factor)
            v_clamped = torch.clamp(exp_avg_sq, min=0.0, max=v_scale * clamp_factor)
        else:
            m_clamped = exp_avg
            v_clamped = exp_avg_sq
        
        # Signal: squared norm of mean gradient (consistent direction)
        signal_per_elem = m_clamped.pow(2)
        
        # Noise: gradient variance = E[g²] - (E[g])²
        noise_per_elem = torch.clamp(v_clamped - signal_per_elem, min=eps)
        
        # Compute mean AGAR across all elements (uniform weighting)
        agar_per_elem = signal_per_elem / (signal_per_elem + noise_per_elem + eps)
        agar = agar_per_elem.mean()
        
        return torch.clamp(agar, min=0.0, max=1.0)
    
    # Calibration constants
    MIN_CALIBRATION_SAMPLES = 50
    MIN_STD_THRESHOLD = 0.01  # Prevent division by zero
    
    # Coefficient of variation thresholds for adaptive c_min
    CV_HIGH_THRESHOLD = 0.5  # Bimodal distribution
    CV_MEDIUM_THRESHOLD = 0.3  # Some separation
    
    # Adaptive c_min values
    C_MIN_HIGH_VARIANCE = 0.1  # Strong discrimination for bimodal
    C_MIN_MEDIUM_VARIANCE = 0.3  # Moderate discrimination
    C_MIN_LOW_VARIANCE = 0.5  # High baseline for unimodal/pervasive noise
    
    def _calibrate_tau(self, agar_buffer: deque, tau_clip_range: Tuple[float, float], group_idx: int) -> float:
        """
        Universal tau calibration using distribution statistics.
        
        Computes distribution parameters for confidence mapping:
        - μ (mean): Central tendency of AGAR distribution
        - σ (std): Spread of AGAR distribution
        - p50 (median): Robust center estimate
        - p10, p90: Distribution bounds for outlier detection
        
        This approach works universally for:
        - Clean data: High μ, low σ → High confidence baseline
        - Pervasive noise: Low μ, low σ → Adaptive confidence scaling
        - Mixed batches: Medium μ, high σ → Bimodal confidence distribution
        
        Mathematical foundation:
            confidence(agar) = c_min + (1 - c_min) * sigmoid((agar - μ) / σ)
        
        This sigmoid mapping naturally adapts to any distribution shape.
        
        Args:
            agar_buffer: Collection of AGAR samples from initial training steps
            tau_clip_range: Safety bounds for tau (min, max)
            group_idx: Parameter group index for storing calibration results
        
        Returns:
            Calibrated tau threshold (median for robustness)
        """
        if len(agar_buffer) < self.MIN_CALIBRATION_SAMPLES:
            return tau_clip_range[1]
        
        samples = np.array(agar_buffer)
        
        # Distribution statistics
        mu = np.mean(samples)
        sigma = np.std(samples)
        median = np.median(samples)
        p10 = np.percentile(samples, 10)
        p90 = np.percentile(samples, 90)
        
        # Store distribution parameters for confidence mapping
        group_state = self._group_states[group_idx]
        group_state['agar_mean'] = float(mu)
        group_state['agar_std'] = float(max(sigma, self.MIN_STD_THRESHOLD))
        group_state['agar_median'] = float(median)
        group_state['agar_p10'] = float(p10)
        group_state['agar_p90'] = float(p90)
        
        # Adaptive c_min based on coefficient of variation (CV = σ/μ)
        # High CV → Lower c_min (strong discrimination for bimodal distributions)
        # Low CV → Higher c_min (prevent over-suppression for unimodal/pervasive noise)
        cv = sigma / (mu + 1e-8)
        if cv > self.CV_HIGH_THRESHOLD:
            c_min_adaptive = self.C_MIN_HIGH_VARIANCE
        elif cv > self.CV_MEDIUM_THRESHOLD:
            c_min_adaptive = self.C_MIN_MEDIUM_VARIANCE
        else:
            c_min_adaptive = self.C_MIN_LOW_VARIANCE
        
        group_state['c_min'] = float(c_min_adaptive)
        
        self._log(2, f"Calibrated AGAR distribution: μ={mu:.4f}, σ={sigma:.4f}, "
                     f"median={median:.4f}, CV={cv:.4f}, c_min={c_min_adaptive:.2f}")
        
        # Return median as tau (robust to outliers)
        return float(np.clip(median, tau_clip_range[0], tau_clip_range[1]))
    
    @torch.no_grad()
    def step(self, closure: Optional[Callable] = None) -> Optional[float]:
        """
        Perform a single optimization step.
        
        Args:
            closure (callable, optional): A closure that reevaluates the model and returns
                the loss. Optional for most optimizers but required for some (e.g., LBFGS).
        
        Returns:
            Optional[float]: Loss value if closure is provided, None otherwise
        
        Raises:
            RuntimeError: If NaN or Inf gradients are detected
            NotImplementedError: If sparse gradients are encountered
        """
        loss = None
        if closure is not None:
            with torch.enable_grad():
                loss = closure()
        
        self._step_count += 1
        
        for group_idx, group in enumerate(self.param_groups):
            beta1, beta2 = group['betas']
            lr = group['lr']
            eps = group['eps']
            weight_decay = group['weight_decay']
            c_min = group['c_min']
            tau_init_steps = group['tau_init_steps']
            tau_clip_range = group['tau_clip_range']
            granularity = group['granularity']
            agar_clamp_factor = group['agar_clamp_factor']
            
            group_state = self._group_states[group_idx]
            
            # Per-group AGAR mode: compute once for all parameters in group
            if granularity == 'group':
                # Skip group if no parameters have gradients
                valid_params = [p for p in group['params'] if p.grad is not None]
                if not valid_params:
                    continue
                
                all_exp_avg = []
                all_exp_avg_sq = []
                
                # First pass: update momentum and collect states
                for p in group['params']:
                    if p.grad is None:
                        continue
                    
                    self._validate_gradient(p.grad, group_idx)
                    state = self._init_param_state(p)
                    self._update_moments(state, p.grad, beta1, beta2)
                    
                    all_exp_avg.append(state['exp_avg'].flatten())
                    all_exp_avg_sq.append(state['exp_avg_sq'].flatten())
                
                # Compute group-level AGAR using pre-allocated buffers
                if all_exp_avg:
                    # Allocate buffers on first use (amortized across all steps)
                    if group_state['reuse_buffer_exp_avg'] is None:
                        total_params = sum(m.numel() for m in all_exp_avg)
                        device = all_exp_avg[0].device
                        dtype = all_exp_avg[0].dtype
                        group_state['reuse_buffer_exp_avg'] = torch.zeros(total_params, device=device, dtype=dtype)
                        group_state['reuse_buffer_exp_avg_sq'] = torch.zeros(total_params, device=device, dtype=dtype)
                    
                    # Copy moment estimates into buffers (avoids repeated allocations)
                    offset = 0
                    reuse_buffer_exp_avg = group_state['reuse_buffer_exp_avg']
                    reuse_buffer_exp_avg_sq = group_state['reuse_buffer_exp_avg_sq']
                    
                    for m, v in zip(all_exp_avg, all_exp_avg_sq):
                        numel = m.numel()
                        reuse_buffer_exp_avg[offset:offset+numel].copy_(m)
                        reuse_buffer_exp_avg_sq[offset:offset+numel].copy_(v)
                        offset += numel
                    
                    # Compute AGAR on concatenated moments
                    agar = self._compute_agar(
                        reuse_buffer_exp_avg[:offset],
                        reuse_buffer_exp_avg_sq[:offset],
                        eps,
                        agar_clamp_factor
                    )
                    
                    agar_value = agar.item()
                    group_state['current_agar'] = agar_value
                    
                    # Tau calibration and adaptation
                    if not group_state['tau_initialized']:
                        group_state['agar_buffer'].append(agar_value)
                        
                        # Diagnostic logging during calibration
                        if self._step_count % 10 == 0 and len(group_state['agar_buffer']) > 0:
                            agars = list(group_state['agar_buffer'])
                            self._log(3, f"Step {self._step_count} - AGAR: min={min(agars):.4f}, median={np.median(agars):.4f}, max={max(agars):.4f}")
                        
                        if len(group_state['agar_buffer']) >= tau_init_steps:
                            tau = self._calibrate_tau(group_state['agar_buffer'], tau_clip_range, group_idx)
                            group_state['tau_adapter'].tau = tau
                            group_state['tau_adapter'].tau_calibrated = tau  # Anchor dead zone to calibrated value
                            group_state['tau_initialized'] = True
                            group_state['agar_buffer'].clear()
                            self._log(2, f"Group {group_idx}: Tau calibrated to {tau:.4f} from {tau_init_steps} samples")
                    else:
                        # Post-calibration: adapt tau using drift-detecting EMA
                        group_state['tau_adapter'].update(agar_value)
                    
                    # Universal sigmoid-based confidence mapping
                    if group_state['tau_initialized']:
                        mu = group_state.get('agar_mean', agar_value)
                        sigma = group_state.get('agar_std', 0.1)
                        c_min_adaptive = group_state.get('c_min', c_min)
                        
                        # Sigmoid mapping: confidence = c_min + (1 - c_min) * sigmoid((agar - μ) / σ)
                        # This naturally adapts to any distribution:
                        # - High μ, low σ (clean): Most samples get high confidence
                        # - Low μ, low σ (pervasive noise): Confidence scales smoothly from c_min
                        # - High σ (mixed): Strong discrimination between low/high AGAR
                        z_score = (agar_value - mu) / sigma
                        sigmoid = 1.0 / (1.0 + np.exp(-z_score))
                        confidence_value = c_min_adaptive + (1.0 - c_min_adaptive) * sigmoid
                        
                        confidence_value = float(np.clip(confidence_value, c_min_adaptive, 1.0))
                    else:
                        # Pre-calibration: simple passthrough
                        confidence_value = float(np.clip(agar_value, c_min, 1.0))
                    
                    group_state['current_confidence'] = confidence_value
                    
                    # Diagnostic logging
                    if group_state['tau_initialized'] and self._step_count % 100 == 0:
                        mu = group_state.get('agar_mean', 0)
                        sigma = group_state.get('agar_std', 0)
                        self._log(3, f"Step {self._step_count} - AGAR={agar_value:.4f}, μ={mu:.4f}, "
                                     f"σ={sigma:.4f}, Confidence={confidence_value:.4f}")
                    
                    confidence_tensor = torch.tensor(confidence_value, device=all_exp_avg[0].device, dtype=all_exp_avg[0].dtype)
                else:
                    confidence_tensor = torch.tensor(c_min)
                
                # Apply parameter updates with confidence-scaled learning rate
                for p in group['params']:
                    if p.grad is None:
                        continue
                    
                    self._apply_weight_update(p, self.state[p], lr, weight_decay, 
                                             eps, confidence_tensor, beta1, beta2)
            
            # Per-parameter AGAR mode: compute separately for each parameter
            else:
                for p in group['params']:
                    if p.grad is None:
                        continue
                    
                    self._validate_gradient(p.grad, group_idx)
                    state = self._init_param_state(p)
                    self._update_moments(state, p.grad, beta1, beta2)
                    
                    # Compute per-parameter AGAR
                    agar = self._compute_agar(state['exp_avg'], state['exp_avg_sq'], eps, agar_clamp_factor)
                    
                    agar_value = agar.item()
                    
                    # Store first parameter's AGAR as representative for monitoring
                    if group_state['current_agar'] is None:
                        group_state['current_agar'] = agar_value
                    
                    # Tau calibration and adaptation
                    if not group_state['tau_initialized']:
                        group_state['agar_buffer'].append(agar_value)
                        
                        # Diagnostic logging during calibration
                        if self._step_count % 10 == 0 and len(group_state['agar_buffer']) > 0:
                            agars = list(group_state['agar_buffer'])
                            self._log(3, f"Step {self._step_count} - AGAR: min={min(agars):.4f}, median={np.median(agars):.4f}, max={max(agars):.4f}")
                        
                        if len(group_state['agar_buffer']) >= tau_init_steps:
                            tau = self._calibrate_tau(group_state['agar_buffer'], tau_clip_range, group_idx)
                            group_state['tau_adapter'].tau = tau
                            group_state['tau_adapter'].tau_calibrated = tau  # Anchor dead zone to calibrated value
                            group_state['tau_initialized'] = True
                            group_state['agar_buffer'].clear()
                            self._log(2, f"Group {group_idx}: Tau calibrated to {tau:.4f} from {tau_init_steps} samples")
                    else:
                        # Post-calibration: adapt tau using drift-detecting EMA
                        group_state['tau_adapter'].update(agar_value)
                    
                    # Universal sigmoid-based confidence mapping
                    if group_state['tau_initialized']:
                        mu = group_state.get('agar_mean', agar_value)
                        sigma = group_state.get('agar_std', 0.1)
                        c_min_adaptive = group_state.get('c_min', c_min)
                        
                        # Sigmoid mapping: confidence = c_min + (1 - c_min) * sigmoid((agar - μ) / σ)
                        z_score = (agar_value - mu) / sigma
                        sigmoid = 1.0 / (1.0 + np.exp(-z_score))
                        confidence_value = c_min_adaptive + (1.0 - c_min_adaptive) * sigmoid
                        
                        confidence_value = float(np.clip(confidence_value, c_min_adaptive, 1.0))
                    else:
                        # Pre-calibration: simple passthrough
                        confidence_value = float(np.clip(agar_value, c_min, 1.0))
                    
                    confidence_tensor = torch.tensor(confidence_value, device=p.device, dtype=p.dtype)
                    
                    # Store first parameter's confidence as representative for monitoring
                    if group_state['current_confidence'] is None:
                        group_state['current_confidence'] = confidence_value
                    
                    # Diagnostic logging
                    if group_state['tau_initialized'] and self._step_count % 100 == 0:
                        mu = group_state.get('agar_mean', 0)
                        sigma = group_state.get('agar_std', 0)
                        self._log(3, f"Step {self._step_count} - AGAR={agar_value:.4f}, μ={mu:.4f}, "
                                     f"σ={sigma:.4f}, Confidence={confidence_value:.4f}")
                    
                    # Apply parameter update
                    self._apply_weight_update(p, state, lr, weight_decay, 
                                             eps, confidence_tensor, beta1, beta2)
        
        return loss
    
    def state_dict(self):
        """
        Return the optimizer state as a dictionary.
        
        Includes all parameter states, hyperparameters, and internal calibration data.
        Compatible with torch.save() for checkpointing.
        
        Returns:
            dict: Complete optimizer state including:
                - Parameter-specific states (exp_avg, exp_avg_sq, step)
                - Group-level calibration data (tau, agar_buffer)
                - Global step counter
        """
        state_dict = super().state_dict()
        
        # Serialize group states (convert deque to list)
        serializable_group_states = {}
        for idx, gs in self._group_states.items():
            serializable_group_states[idx] = {
                'tau_initialized': gs['tau_initialized'],
                'agar_buffer': list(gs['agar_buffer']),
                'agar_buffer_maxlen': gs['agar_buffer'].maxlen,
                'adapter_tau': gs['tau_adapter'].tau,
                'adapter_tau_calibrated': gs['tau_adapter'].tau_calibrated,
                'adapter_mean': gs['tau_adapter'].mean_agar,
                'adapter_var': gs['tau_adapter'].ema_var,
                'agar_mean': gs.get('agar_mean'),
                'agar_std': gs.get('agar_std'),
                'agar_median': gs.get('agar_median'),
                'agar_p10': gs.get('agar_p10'),
                'agar_p90': gs.get('agar_p90'),
                'c_min': gs.get('c_min'),
            }
        
        state_dict['_group_states'] = serializable_group_states
        state_dict['_step_count'] = self._step_count
        
        return state_dict
    
    def load_state_dict(self, state_dict):
        """
        Load optimizer state from a dictionary.
        
        Restores all parameter states, hyperparameters, and internal calibration data.
        Compatible with torch.load() for checkpoint restoration.
        
        Args:
            state_dict (dict): Optimizer state dictionary (typically from state_dict())
        
        Note:
            Automatically handles conversion of serialized lists back to deque objects
            for AGAR buffer management.
        """
        # Restore group states (convert list back to deque)
        if '_group_states' in state_dict:
            loaded_states = state_dict.pop('_group_states')
            # Ensure keys are integers (they may be strings after JSON serialization)
            self._group_states = {}
            for idx, gs in loaded_states.items():
                idx_int = int(idx) if isinstance(idx, str) else idx
                
                maxlen = gs.pop('agar_buffer_maxlen', None)
                buffer_list = gs.pop('agar_buffer', [])
                gs['agar_buffer'] = deque(buffer_list, maxlen=maxlen)
                
                # Restore adapter state
                tau_clip_range = self.param_groups[idx_int]['tau_clip_range']
                tau_dead_zone = self.param_groups[idx_int]['tau_dead_zone']
                adapter = DDEAdapter(1.0, tau_clip_range, dead_zone_factor=tau_dead_zone)
                adapter.tau = gs.pop('adapter_tau', 1.0)
                adapter.tau_calibrated = gs.pop('adapter_tau_calibrated', None)
                adapter.mean_agar = gs.pop('adapter_mean', 1.0)
                adapter.ema_var = gs.pop('adapter_var', 0.01)
                gs['tau_adapter'] = adapter
                
                # Initialize missing fields
                gs.setdefault('reuse_buffer_exp_avg', None)
                gs.setdefault('reuse_buffer_exp_avg_sq', None)
                gs.setdefault('current_agar', None)
                gs.setdefault('current_confidence', None)
                gs.setdefault('agar_mean', None)
                gs.setdefault('agar_std', None)
                gs.setdefault('agar_median', None)
                gs.setdefault('agar_p10', None)
                gs.setdefault('agar_p90', None)
                gs.setdefault('c_min', self.param_groups[idx_int].get('c_min', 0.1))
                
                self._group_states[idx_int] = gs
        
        # Restore step count
        if '_step_count' in state_dict:
            self._step_count = state_dict.pop('_step_count')
        
        super().load_state_dict(state_dict)

In [ ]:
"""
Long-Tailed Recognition Benchmark: CIFAR-100 with Class Imbalance

Tests CASMO's ability to handle imbalanced datasets where standard optimizers
overfit to majority classes and ignore minority (tail) classes.

Task:
    Dataset: CIFAR-100 with exponential long-tail distribution
    Imbalance Ratio: 100:1 (most frequent class has 100x samples of rarest)
    
    Hypothesis:
    - AdamW will overfit to majority classes (high head accuracy, low tail accuracy).
    - CASMO will balance learning across all classes (better tail accuracy).
    
    Reasoning:
    - Majority class gradients: Consistent (high AGAR) → normal learning rate
    - Minority class gradients: Noisy due to few samples (low AGAR) → CASMO prevents overfitting
"""

import sys
import os
import argparse

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torchvision
import torchvision.transforms as transforms
import time
import random
import numpy as np 
import matplotlib.pyplot as plt
from pathlib import Path
from collections import defaultdict


# -----------------------------------------------------------------------------
# 1. Dataset: Long-Tailed CIFAR-100
# -----------------------------------------------------------------------------

class LongTailCIFAR100(Dataset):
    """
    CIFAR-100 with exponential long-tail distribution.
    
    Creates imbalanced dataset where:
    - Head classes (many-shot): >100 samples
    - Medium classes (medium-shot): 20-100 samples  
    - Tail classes (few-shot): <20 samples
    """
    def __init__(self, root='./data', train=True, imbalance_factor=100, seed=42):
        self.train = train
        self.imbalance_factor = imbalance_factor
        
        # Load full CIFAR-100
        self.dataset = torchvision.datasets.CIFAR100(
            root=root, 
            train=train, 
            download=True,
            transform=None  # We'll apply transforms later
        )
        
        if train:
            # Create long-tail distribution
            self.data, self.targets, self.class_counts = self._create_long_tail(seed)
        else:
            # Keep test set balanced
            self.data = self.dataset.data
            self.targets = self.dataset.targets
            self.class_counts = self._count_classes()
        
        # Define transforms
        if train:
            self.transform = transforms.Compose([
                transforms.ToPILImage(),
                transforms.RandomCrop(32, padding=4),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
            ])
        else:
            self.transform = transforms.Compose([
                transforms.ToPILImage(),
                transforms.ToTensor(),
                transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
            ])
    
    def _create_long_tail(self, seed):
        """Create exponentially imbalanced dataset."""
        np.random.seed(seed)
        
        num_classes = 100
        # Get samples per class in original dataset
        original_data = np.array(self.dataset.data)
        original_targets = np.array(self.dataset.targets)
        
        # Calculate target samples per class (exponential decay)
        max_samples = 500  # Maximum samples for most frequent class
        samples_per_class = []
        for i in range(num_classes):
            # Exponential decay: n_i = n_max * (imb_factor)^(-i/(C-1))
            n_samples = int(max_samples * (self.imbalance_factor ** (-i / (num_classes - 1))))
            samples_per_class.append(max(n_samples, 5))  # At least 5 samples per class
        
        # Sample from each class
        new_data = []
        new_targets = []
        class_counts = {}
        
        for class_idx in range(num_classes):
            class_mask = original_targets == class_idx
            class_data = original_data[class_mask]
            
            n_samples = samples_per_class[class_idx]
            n_available = len(class_data)
            
            if n_samples > n_available:
                # Oversample if needed
                indices = np.random.choice(n_available, n_samples, replace=True)
            else:
                # Subsample
                indices = np.random.choice(n_available, n_samples, replace=False)
            
            sampled_data = class_data[indices]
            new_data.append(sampled_data)
            new_targets.extend([class_idx] * n_samples)
            class_counts[class_idx] = n_samples
        
        new_data = np.concatenate(new_data, axis=0)
        new_targets = np.array(new_targets)
        
        # Shuffle
        shuffle_idx = np.random.permutation(len(new_targets))
        new_data = new_data[shuffle_idx]
        new_targets = new_targets[shuffle_idx]
        
        return new_data, new_targets.tolist(), class_counts
    
    def _count_classes(self):
        """Count samples per class."""
        class_counts = {}
        for target in self.targets:
            class_counts[target] = class_counts.get(target, 0) + 1
        return class_counts
    
    def get_class_groups(self):
        """Categorize classes into many/medium/few shot."""
        many_shot = []  # >100 samples
        medium_shot = []  # 20-100 samples
        few_shot = []  # <20 samples
        
        for class_idx, count in self.class_counts.items():
            if count > 100:
                many_shot.append(class_idx)
            elif count >= 20:
                medium_shot.append(class_idx)
            else:
                few_shot.append(class_idx)
        
        return many_shot, medium_shot, few_shot
    
    def __len__(self):
        return len(self.targets)
    
    def __getitem__(self, idx):
        img = self.data[idx]
        target = self.targets[idx]
        
        if self.transform:
            img = self.transform(img)
        
        return img, target


# -----------------------------------------------------------------------------
# 2. Model: ResNet-32 for CIFAR-100
# -----------------------------------------------------------------------------

class BasicBlock(nn.Module):
    """Basic residual block for ResNet."""
    expansion = 1
    
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    """ResNet for CIFAR-100."""
    def __init__(self, block, num_blocks, num_classes=100):
        super(ResNet, self).__init__()
        self.in_planes = 16
        
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)
        self.linear = nn.Linear(64 * block.expansion, num_classes)
    
    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet32(num_classes=100):
    """ResNet-32 for CIFAR-100."""
    return ResNet(BasicBlock, [5, 5, 5], num_classes=num_classes)


# -----------------------------------------------------------------------------
# 3. Utilities
# -----------------------------------------------------------------------------

def set_seed(seed=42):
    """Set random seeds for reproducibility."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


def get_agar_confidence(optimizer):
    """Extract current AGAR and confidence from CASMO optimizer."""
    if not hasattr(optimizer, '_group_states'):
        return None, None
    group_state = optimizer._group_states.get(0, {})
    return group_state.get('current_agar'), group_state.get('current_confidence')


def compute_per_group_accuracy(model, loader, many_shot_classes, medium_shot_classes, few_shot_classes, device):
    """Compute accuracy for many/medium/few shot classes."""
    model.eval()
    
    many_correct = 0
    many_total = 0
    medium_correct = 0
    medium_total = 0
    few_correct = 0
    few_total = 0
    
    with torch.no_grad():
        for inputs, targets in loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            
            for pred, target in zip(predicted, targets):
                target_item = target.item()
                correct = (pred == target).item()
                
                if target_item in many_shot_classes:
                    many_total += 1
                    many_correct += correct
                elif target_item in medium_shot_classes:
                    medium_total += 1
                    medium_correct += correct
                elif target_item in few_shot_classes:
                    few_total += 1
                    few_correct += correct
    
    many_acc = 100. * many_correct / many_total if many_total > 0 else 0
    medium_acc = 100. * medium_correct / medium_total if medium_total > 0 else 0
    few_acc = 100. * few_correct / few_total if few_total > 0 else 0
    
    return many_acc, medium_acc, few_acc


# -----------------------------------------------------------------------------
# 4. Training Loop
# -----------------------------------------------------------------------------

def run_benchmark(optimizer_name, device, train_loader, test_loader, train_dataset, 
                 num_epochs=200, lr=0.1, weight_decay=5e-4, seed=42):
    
    print(f"\n{'='*70}")
    print(f"Running: {optimizer_name.upper()}")
    print(f"{'='*70}\n")
    
    set_seed(seed)
    
    model = ResNet32(num_classes=100)
    model.to(device)
    
    criterion = nn.CrossEntropyLoss()
    
    if optimizer_name == 'casmo':
        total_steps = len(train_loader) * num_epochs
        tau_init_steps = max(50, int(0.05 * total_steps))
        
        optimizer = CASMO(
            model.parameters(),
            lr=lr,
            weight_decay=weight_decay,
            granularity='group',
            log_level=1,
            tau_init_steps=tau_init_steps,
            tau_dead_zone=0.2,
            c_min=0.1
        )
    else:
        optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    
    # Cosine annealing scheduler
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
    
    # Get class groups for per-group accuracy
    many_shot, medium_shot, few_shot = train_dataset.get_class_groups()
    
    results = {
        'train_accs': [],
        'val_accs': [],
        'train_losses': [],
        'val_losses': [],
        'many_shot_accs': [],
        'medium_shot_accs': [],
        'few_shot_accs': [],
        'agar_values': [],
        'confidence_values': [],
        'steps': []
    }
    
    step = 0
    start_time = time.time()
    
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        correct = 0
        total = 0
        
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
            # Log AGAR and confidence
            agar, confidence = get_agar_confidence(optimizer)
            if agar is not None and step % 10 == 0:
                results['agar_values'].append(agar)
                results['confidence_values'].append(confidence)
                results['steps'].append(step)
            
            step += 1
        
        train_acc = 100. * correct / total
        train_loss /= len(train_loader)
        
        # Validation
        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()
        
        val_acc = 100. * correct / total
        val_loss /= len(test_loader)
        
        # Compute per-group accuracy
        many_acc, medium_acc, few_acc = compute_per_group_accuracy(
            model, test_loader, many_shot, medium_shot, few_shot, device
        )
        
        results['train_accs'].append(train_acc)
        results['val_accs'].append(val_acc)
        results['train_losses'].append(train_loss)
        results['val_losses'].append(val_loss)
        results['many_shot_accs'].append(many_acc)
        results['medium_shot_accs'].append(medium_acc)
        results['few_shot_accs'].append(few_acc)
        
        scheduler.step()
        
        if (epoch + 1) % 20 == 0:
            print(f"Epoch {epoch+1}/{num_epochs} | Train: {train_acc:.1f}% | Val: {val_acc:.1f}% | "
                  f"Many: {many_acc:.1f}% | Medium: {medium_acc:.1f}% | Few: {few_acc:.1f}%")
    
    total_time = time.time() - start_time
    print(f"Finished in {total_time:.1f}s")
    
    # Print final AGAR/Confidence statistics
    if optimizer_name == 'casmo' and results['agar_values']:
        print(f"\n{'='*70}")
        print(f"Final AGAR Statistics (last 100 steps):")
        print(f"{'='*70}")
        agar_final = results['agar_values'][-100:] if len(results['agar_values']) >= 100 else results['agar_values']
        print(f"  Mean: {np.mean(agar_final):.4f}")
        print(f"  Std:  {np.std(agar_final):.4f}")
        print(f"  Min:  {min(agar_final):.4f}")
        print(f"  Max:  {max(agar_final):.4f}")
        
        print(f"\n{'='*70}")
        print(f"Final Confidence Statistics (last 100 steps):")
        print(f"{'='*70}")
        conf_final = results['confidence_values'][-100:] if len(results['confidence_values']) >= 100 else results['confidence_values']
        print(f"  Mean: {np.mean(conf_final):.4f}")
        print(f"  Std:  {np.std(conf_final):.4f}")
        print(f"  Min:  {min(conf_final):.4f}")
        print(f"  Max:  {max(conf_final):.4f}")
        print(f"{'='*70}\n")
    
    return results


# -----------------------------------------------------------------------------
# 5. Main Execution
# -----------------------------------------------------------------------------

def main():
    
    print("="*70)
    print(f"CASMO vs AdamW Benchmark - Long-Tailed CIFAR-100")
    print(f"Imbalance Factor: {100}:1")
    print("="*70)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Device: {device}")
    
    # Setup Data
    train_dataset = LongTailCIFAR100(root='./data', train=True, imbalance_factor=100)
    test_dataset = LongTailCIFAR100(root='./data', train=False, imbalance_factor=100)
    
    # Print dataset statistics
    many_shot, medium_shot, few_shot = train_dataset.get_class_groups()
    print(f"\nDataset Statistics:")
    print(f"  Total training samples: {len(train_dataset)}")
    print(f"  Many-shot classes (>100 samples): {len(many_shot)}")
    print(f"  Medium-shot classes (20-100 samples): {len(medium_shot)}")
    print(f"  Few-shot classes (<20 samples): {len(few_shot)}")
    
    train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2)
    
    # Run Benchmarks
    print("\nStarting CASMO run...")
    casmo_results = run_benchmark('casmo', device, train_loader, test_loader, train_dataset,
                                  num_epochs=200, lr=0.1, weight_decay=5e-4)
    
    print("\nStarting AdamW run...")
    adamw_results = run_benchmark('adamw', device, train_loader, test_loader, train_dataset,
                                  num_epochs=200, lr=0.1, weight_decay=5e-4)
    
    # Plotting
    print("\nPlotting results...")
    
    # Create results directory if it doesn't exist
    results_dir = "results"
    os.makedirs(results_dir, exist_ok=True)
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. Overall Accuracy Curves
    axes[0, 0].plot(casmo_results['train_accs'], label='CASMO Train', linestyle='--', alpha=0.5, color='green')
    axes[0, 0].plot(casmo_results['val_accs'], label='CASMO Val', linewidth=2, color='green')
    axes[0, 0].plot(adamw_results['train_accs'], label='AdamW Train', linestyle='--', alpha=0.5, color='orange')
    axes[0, 0].plot(adamw_results['val_accs'], label='AdamW Val', linewidth=2, color='orange')
    axes[0, 0].set_title(f'Overall Accuracy (Imbalance: {args.imbalance}:1)')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Accuracy (%)')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Per-Group Accuracy
    epochs = range(len(casmo_results['many_shot_accs']))
    axes[0, 1].plot(epochs, casmo_results['many_shot_accs'], label='CASMO Many-shot', color='green', linestyle='-', linewidth=2)
    axes[0, 1].plot(epochs, casmo_results['medium_shot_accs'], label='CASMO Medium-shot', color='green', linestyle='--', linewidth=2)
    axes[0, 1].plot(epochs, casmo_results['few_shot_accs'], label='CASMO Few-shot', color='green', linestyle=':', linewidth=2)
    axes[0, 1].plot(epochs, adamw_results['many_shot_accs'], label='AdamW Many-shot', color='orange', linestyle='-', linewidth=2, alpha=0.7)
    axes[0, 1].plot(epochs, adamw_results['medium_shot_accs'], label='AdamW Medium-shot', color='orange', linestyle='--', linewidth=2, alpha=0.7)
    axes[0, 1].plot(epochs, adamw_results['few_shot_accs'], label='AdamW Few-shot', color='orange', linestyle=':', linewidth=2, alpha=0.7)
    axes[0, 1].set_title('Per-Group Accuracy')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Accuracy (%)')
    axes[0, 1].legend(fontsize=8)
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. AGAR Evolution (CASMO only)
    if casmo_results['agar_values']:
        axes[1, 0].plot(casmo_results['steps'], casmo_results['agar_values'], color='green', alpha=0.6)
        axes[1, 0].set_title('CASMO: AGAR Signal Strength')
        axes[1, 0].set_xlabel('Step')
        axes[1, 0].set_ylabel('AGAR')
        axes[1, 0].grid(True, alpha=0.3)
    
    # 4. Confidence Evolution (CASMO only)
    if casmo_results.get('confidence_values'):
        axes[1, 1].plot(casmo_results['steps'], casmo_results['confidence_values'], color='blue', alpha=0.6)
        axes[1, 1].set_title('CASMO: Confidence Score')
        axes[1, 1].set_xlabel('Step')
        axes[1, 1].set_ylabel('Confidence')
        axes[1, 1].set_ylim(0, 1.05)
        axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    save_path = os.path.join(results_dir, 'long_tail_metrics.png')
    plt.savefig(save_path, dpi=150)
    print(f"✅ Plot saved to {save_path}")
    
    # Print final comparison
    print(f"\n{'='*70}")
    print(f"FINAL RESULTS COMPARISON")
    print(f"{'='*70}")
    print(f"{'Metric':<30} {'CASMO':>15} {'AdamW':>15}")
    print(f"{'-'*70}")
    print(f"{'Overall Val Accuracy':<30} {casmo_results['val_accs'][-1]:>14.2f}% {adamw_results['val_accs'][-1]:>14.2f}%")
    print(f"{'Many-shot Accuracy':<30} {casmo_results['many_shot_accs'][-1]:>14.2f}% {adamw_results['many_shot_accs'][-1]:>14.2f}%")
    print(f"{'Medium-shot Accuracy':<30} {casmo_results['medium_shot_accs'][-1]:>14.2f}% {adamw_results['medium_shot_accs'][-1]:>14.2f}%")
    print(f"{'Few-shot Accuracy':<30} {casmo_results['few_shot_accs'][-1]:>14.2f}% {adamw_results['few_shot_accs'][-1]:>14.2f}%")
    print(f"{'='*70}\n")


if __name__ == '__main__':
    main()


CASMO vs AdamW Benchmark - Long-Tailed CIFAR-100
Imbalance Factor: 100:1
Device: cuda


100%|██████████| 169M/169M [00:05<00:00, 30.4MB/s] 



Dataset Statistics:
  Total training samples: 10847
  Many-shot classes (>100 samples): 35
  Medium-shot classes (20-100 samples): 35
  Few-shot classes (<20 samples): 30

Starting CASMO run...

Running: CASMO

Epoch 20/200 | Train: 36.6% | Val: 15.8% | Many: 34.3% | Medium: 9.7% | Few: 1.2%
Epoch 40/200 | Train: 57.2% | Val: 24.8% | Many: 48.1% | Medium: 19.0% | Few: 4.6%
Epoch 60/200 | Train: 71.6% | Val: 25.4% | Many: 48.1% | Medium: 21.5% | Few: 3.7%
Epoch 80/200 | Train: 82.3% | Val: 27.8% | Many: 51.8% | Medium: 23.5% | Few: 4.9%
Epoch 100/200 | Train: 88.3% | Val: 27.0% | Many: 50.6% | Medium: 21.5% | Few: 5.7%
Epoch 120/200 | Train: 93.8% | Val: 28.2% | Many: 51.3% | Medium: 24.9% | Few: 5.2%
Epoch 140/200 | Train: 96.8% | Val: 28.2% | Many: 52.4% | Medium: 22.2% | Few: 6.9%
Epoch 160/200 | Train: 98.3% | Val: 28.9% | Many: 53.2% | Medium: 24.2% | Few: 6.1%
Epoch 180/200 | Train: 99.2% | Val: 29.1% | Many: 53.1% | Medium: 24.7% | Few: 6.1%
Epoch 200/200 | Train: 99.5% | Val: 2

NameError: name '__file__' is not defined